In [1]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv3D, Activation, Dense, GlobalAveragePooling3D, MaxPooling3D, ZeroPadding3D, Add
from keras.optimizers import Adam 
import os
#import matplotlib.pyplot as plt
import numpy as np
import math

import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/old_germination_data.npy')

In [4]:
train_x.shape

(1536, 60, 60, 275, 1)

In [5]:
t1 = train_x[286:1536,:,:,20:160]
tt1 = train_x[0:286,:,:,20:160]

In [6]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/germination_data/label/old_germination_label.csv")
label_df.head(5)

,label
0,0
1,0
2,0
3,1
4,0


In [7]:
from keras.utils import to_categorical
train_y = label_df[286:1536]
train_y = to_categorical(train_y)

In [8]:
test_y = label_df[0:286]
test_y = to_categorical(test_y)

In [9]:
t1 = t1.astype('float32')
tt1 = tt1.astype('float32')

In [10]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [11]:
input_tensor = Input(shape=(60, 60, 140,1), dtype='float32', name='input')

W1023 22:27:09.351043 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [12]:
def conv1_layer(x):    
    #x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv3D(64, (7, 7, 7), strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding3D(padding=(1,1,1))(x)
 
    return x   
 
    
def conv2_layer(x):         
    x = MaxPooling3D((2, 2, 2), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv3D(64, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv3D(64, (3, 3,3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv3D(256, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            shortcut = Conv3D(256, (1, 1,1), strides=1, padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv3D(64, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv3D(64, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv3D(256, (1, 1,1 ), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x
 
 
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv3D(128, (1, 1, 1), strides=(2, 2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv3D(128, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv3D(512, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            shortcut = Conv3D(512, (1, 1,1), strides=(2, 2,2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv3D(128, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv3D(128, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv3D(512, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv3D(256, (1, 1, 1), strides=(2, 2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv3D(256, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv3D(1024, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            shortcut = Conv3D(1024, (1, 1, 1), strides=(2, 2,2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv3D(256, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv3D(256, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv3D(1024, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x
 
 
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv3D(512, (1, 1, 1), strides=(2, 2,2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv3D(512, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv3D(2048, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            shortcut = Conv3D(2048, (1, 1, 1), strides=(2, 2,2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv3D(512, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv3D(512, (3, 3, 3), strides=(1, 1,1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv3D(2048, (1, 1, 1), strides=(1, 1,1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 

In [13]:
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling3D()(x)
output_tensor = Dense(2, activation='softmax')(x)

W1023 22:27:09.414879 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1023 22:27:09.416657 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1023 22:27:09.506452 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



In [14]:
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 60, 60, 140,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 27, 27, 67, 6 22016       input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 27, 27, 67, 6 256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 27, 27, 67, 6 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [15]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model,gpus=2)

W1023 22:27:15.503031 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [16]:
adam = Adam(lr=0.0001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W1023 22:27:28.577417 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [17]:
filepath = "select_resnet.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='select_resnet',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [18]:
import timeit
start = timeit.default_timer()

history = model.fit(x=t1, y=train_y , batch_size=32, epochs=200,validation_split=0.1 ,callbacks=callbacks_list)

stop = timeit.default_timer()
print(stop - start)

W1023 22:27:30.031866 139784099845952 deprecation.py:323] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1125 samples, validate on 125 samples


W1023 22:28:01.182326 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1120: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W1023 22:28:01.565701 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1023 22:28:01.576318 139784099845952 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/200
1125/1125 [==============================] - 56s 50ms/step - loss: 0.7877 - acc: 0.5520 - val_loss: 5.6475 - val_acc: 0.6480

Epoch 00001: val_acc improved from -inf to 0.64800, saving model to select_resnet.hdf5
Epoch 2/200
1125/1125 [==============================] - 20s 18ms/step - loss: 0.7295 - acc: 0.5404 - val_loss: 0.7216 - val_acc: 0.6000

Epoch 00002: val_acc did not improve from 0.64800
Epoch 3/200
1125/1125 [==============================] - 20s 18ms/step - loss: 0.7191 - acc: 0.5884 - val_loss: 1.2971 - val_acc: 0.3520

Epoch 00003: val_acc did not improve from 0.64800
Epoch 4/200
1125/1125 [==============================] - 20s 18ms/step - loss: 0.7108 - acc: 0.5804 - val_loss: 0.8622 - val_acc: 0.6480

Epoch 00004: val_acc did not improve from 0.64800
Epoch 5/200
1125/1125 [==============================] - 20s 18ms/step - loss: 0.6550 - acc: 0.6293 - val_loss: 1.1020 - val_acc: 0.3520

Epoch 00005: val_acc did not improve from 0.64800
Epoch 6/200
1125/1125 [

In [19]:
# load best weights
model.load_weights("select_resnet.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [20]:
#model.predict(test_x,batch_size=10)
model.evaluate(tt1,test_y,batch_size=32)

286/286 [==============================] - 6s 22ms/step


[0.6680759900099748, 0.6748251750335826]